## OpenCV Face Recognizers

In [16]:
#import OpenCV module
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

In [17]:
#Eğitim verilerimizde 0 etiketi bulunmadığından dizin/etiket 0 için konu adı boş.
subjects = ["","Mehmet Goksu"]

In [18]:
#yüz algılama için için opencv fonksiyonu
def detect_face(img):
    #opencv yüz dedektörü gri görüntüler beklediğinden test görüntüsünü gri görüntüye dönüştürdük
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
#hızlı olduğu için LBD sınıflandırma dedektörünü kullanıyorum
    face_cascade = cv2.CascadeClassifier('opencv-files/lbpcascade_frontalface.xml')

#birden fazla yüz şeklinin olduğu görüntüler ile test ediyoruz çıkan sonuç yüzlerin listesi
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    
    #yüz bulamazsa orijinal hale geri getiriyoruz
    if (len(faces) == 0):
        return None, None
    
#yüz bölgesini çıkartıyoruz
    (x, y, w, h) = faces[0]
    
    #sadece resimin yüz kısmını döndürüyoruz
    return gray[y:y+w, x:x+h], faces[0]

In [19]:
#bu fonksiyon eğitim için gerekli resimleri okuyacak ve onları yüzler ve etiketler olarak ayıracak
def prepare_training_data(data_folder_path):
    

    dirs = os.listdir(data_folder_path)
    faces = []
    labels = []
    
    for dir_name in dirs:
        
        #s ile başlayan klasörleri geziyor
        if not dir_name.startswith("s"):
            continue;
            
        #şimdi dosyalardaki s harfini kaldırıyoruz
        label = int(dir_name.replace("s", ""))
        
#sample ları alıyoruz
        subject_dir_path = data_folder_path + "/" + dir_name
        
#klasördeki resimleri çekiyoruz
        subject_images_names = os.listdir(subject_dir_path)
        
#her resimdeki yüzleri bulup ayıklıyoruz
        for image_name in subject_images_names:
            
            #sistem dosyalarını ignore luyoruz
            if image_name.startswith("."):
                continue;
            
#resim yolu oluşturuyoruz
            image_path = subject_dir_path + "/" + image_name

#resmi okuyoruz
            image = cv2.imread(image_path)
            
#okunan resimleri kısa süreliğine ekrana yansıtıyoruz
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)
            
#yüz tespit ediyoruz
            face, rect = detect_face(image)
            
#yüz yoksa ignoreluyoruz
            if face is not None:
                #add face to list of faces
                faces.append(face)
                #add label for this face
                labels.append(label)
            
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels

In [20]:
#şimdi eğitim datamızı hazırlıyoruz
print("Preparing data...")
faces, labels = prepare_training_data("training-data")
print("Data prepared")


print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

Preparing data...
Data prepared
Total faces:  26
Total labels:  26


In [21]:
#LBPH yüz tanıyıcımızı oluşturuyoruz
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

#diğer recognizerlerimizde mevcut
#face_recognizer = cv2.face_createEigenFaceRecognizer()

#face_recognizer = cv2.face.createFisherFaceRecognizer()

In [22]:
#şimdi train verisi ile eğitiyoruz
face_recognizer.train(faces, np.array(labels))

In [23]:
#resimde kare çizdirme işlemi yapıyoruz
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
#label daki ismi ekrana yazdırıyoruz
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [29]:
error1_count=0
correct1_count=0
def decr():
    global error1_count
    error1_count+=1
def incr():
    global correct1_count
    correct1_count+=1

def predict(test_img):
# Görüntünün bir kopyasını oluşturuyoruz
    img = test_img.copy()
# Yüzü tespit ediyoruz
    face, rect = detect_face(img)
# Yüz tespit edildiyse
    if face is not None:
# Yüzü uygun boyuta getiriyoruz (örneğin, 200x200)
        face = cv2.resize(face, (200, 200))
        
# Yüzü tanıma işlemi yaptırıyoruz
        try:
            label, confidence = face_recognizer.predict(face)
# Label'ın adını al
            label_text = subjects[label]
            incr()
        except Exception as e:
# Hata durumunda varsayılan bir etiket kullan
            print("Yüz tanıma sırasında bir hata oluştu:", e)
            label_text = "Tanimlanamiyor"
        
# Yüzün etrafına bir dikdörtgen çiz
        draw_rectangle(img, rect)
# Tahmin edilen kişinin(label) adını yaz
        draw_text(img, label_text, rect[0], rect[1] - 5)
    else:
        label_text = "Yuz tespit edilemedi"
        decr()
        draw_text(img, label_text, 10, 30)  
    
    return img

In [30]:

print(correct1_count)
print(error1_count)

f_cas= cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
eye_cascade=cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_eye.xml')
#espden görüntü almaya başlıyoruz
url='http://192.168.1.105/cam-hi.jpg'
#bunun gibi farklı çözünürlük kullanabiliriz #'''cam.bmp / cam-lo.jpg /cam-hi.jpg / cam.mjpeg '''
#ekran oluşturuyoruz
cv2.namedWindow("Live Transmission", cv2.WINDOW_AUTOSIZE)
cycle=0
dongu=0
while True:
#urldeki görüntüden her bir frame yi okuyoruz çünkü esp miz istek atılınca frame gönderecek şekilde ayarlı    
#bu yüzden while true döngüsü kullandım    
    img_resp=urllib.request.urlopen(url)
    imgnp=np.array(bytearray(img_resp.read()),dtype=np.uint8)
    img=cv2.imdecode(imgnp,-1)
#resmi gri hale getiriyoruz yüz tanıması için
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#yüz algılıyoruz
    face=f_cas.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5)
#yüz algılanırsa döngüye giriliyor
    for x,y,w,h in face:
        
        
       # cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
#resime renk dönüşümü yapılıyor
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
#resim tahmin ettiriliyor
        predicted_img1 = predict(img)
#bu kısım büyük çözünürlüklü resimler(tahmin edilen predictedlar) olaki algılanırsa ve ekranda taşarsa diye sabit olarak bir çözünürlüğe hapsediliyor
        screen_res = 1920, 1080  # Ekran çözünürlüğü (örnek olarak)
        scale_width = screen_res[0] / predicted_img1.shape[1]
        scale_height = screen_res[1] / predicted_img1.shape[0]
        scale = min(scale_width, scale_height)

        window_width = int(predicted_img1.shape[1] * scale)
        window_height = int(predicted_img1.shape[0] * scale)

        dim = (window_width, window_height)
        img = cv2.resize(predicted_img1, dim, interpolation=cv2.INTER_AREA)
        dongu=dongu+1
#burada espden gelen resim belirli çözünürlüğe getiriliyor      
    screen_res = 1920, 1080  
    scale_width = screen_res[0] / img.shape[1]
    scale_height = screen_res[1] / img.shape[0]
    scale = min(scale_width, scale_height)

    window_width = int(img.shape[1] * scale)
    window_height = int(img.shape[0] * scale)

    dim = (window_width, window_height)
    img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
#burada her 100 framede kaç adet yüz recog edilmiş ne kadar edilmemiş ona bakılarak espye
#bilgi gönderiliyor
    if dongu==100:
        print(correct1_count)
        print(error1_count)
        if correct1_count<error1_count:
#eğer tanımlanmayan yüz varsa 12 ona göre espye hangi pinlerin kapanması gerektiğini
#söyleyen http isteği atıyoruz
            URL1 = "http://192.168.1.105/14/off"
            r = requests.get(url = URL1)
            URL = "http://192.168.1.105/12/on"
            r1 = requests.get(url = URL)
        else:
#burada ise tanımlı olan varsa bu bilgi esp32 ye gönderiliyor
            URL1 = "http://192.168.1.105/14/on"
            r = requests.get(url = URL1)
            URL = "http://192.168.1.105/12/off"
            r1 = requests.get(url = URL)
        correct1_count=0
        error1_count=0
        dongu=0
    cv2.imshow("live transmission",img)
    key=cv2.waitKey(5)
#q ya basılarak uygulamadan çıkılabiliyor
    if key==ord('q'):
        break
cv2.destroyAllWindows()
#burada esp uykuya geçmeden kod kapanırsa espnin ledleri söndürmesi için bilgi gönderiyoruz
URL = "http://192.168.1.105/12/off"
URL1 = "http://192.168.1.105/14/off"
r = requests.get(url = URL1)
r = requests.get(url = URL)

0
0
55
45
iç
2
98
iç
36
64
iç
18
82
iç
39
61
iç
57
43
iç
41
59
iç
11
89
iç
